In [ ]:
import cv2
import numpy as np
import time
import torch
import torchvision.models as models
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity # (Available, but BFMatcher is used)

# ============ PCA-Deep-SIFT-HOG Descriptor ============
class DeepHOGDescriptor:
    def __init__(self, n_components=3, patch_size=64, device="cuda"):
        self.device = device if torch.cuda.is_available() else "cpu"
        print(f"DeepDescriptor using device: {self.device}")
        self.model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).features[:16].to(self.device).eval()
        self.pca = PCA(n_components=n_components)
        self.patch_size = patch_size
        self.trained = False

    def fit_pca(self, gray_images):
        patches = []
        for g in gray_images:
            h, w = g.shape
            if h <= self.patch_size or w <= self.patch_size:
                print(f"Skipping frame in fit_pca, too small: {g.shape}")
                continue
                
            for _ in range(20):  # sample 20 random patches per frame
                try:
                    x = np.random.randint(self.patch_size // 2, w - self.patch_size // 2)
                    y = np.random.randint(self.patch_size // 2, h - self.patch_size // 2)
                    patch = g[y - self.patch_size // 2: y + self.patch_size // 2,
                              x - self.patch_size // 2: x + self.patch_size // 2]
                    patches.append(patch)
                except ValueError:
                    print(f"Warning: Could not sample patch from frame of size {g.shape}")
                    break
        
        if not patches:
            print("⚠️ Warning: No patches sampled to train PCA. Descriptor may fail.")
            return

        feats = []
        for p in patches:
            p_resized = cv2.resize(p, (self.patch_size, self.patch_size))
            t = torch.tensor(p_resized / 255.).float().unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1).to(self.device)
            
            with torch.inference_mode():
                fmap = self.model(t).detach().cpu().numpy()[0]
            
            feats.append(fmap.reshape(fmap.shape[0], -1).mean(axis=1))

        if feats:
            self.pca.fit(np.vstack(feats))
            self.trained = True
            print(f"✅ PCA fitted on {len(patches)} random sample patches.")
        else:
            print("⚠️ Warning: Could not extract features to train PCA.")

    def extract(self, gray_patch):
        if not self.trained:
            return None
        
        if gray_patch.shape[0] != self.patch_size or gray_patch.shape[1] != self.patch_size:
            gray_patch = cv2.resize(gray_patch, (self.patch_size, self.patch_size))

        t = torch.tensor(gray_patch / 255.).float().unsqueeze(0).unsqueeze(0).repeat(1, 3, 1, 1).to(self.device)
        
        with torch.inference_mode():
            fmap = self.model(t).detach().cpu().numpy()[0]

        fmap_flat = fmap.reshape(fmap.shape[0], -1).T
        
        try:
            fmap_pca = self.pca.transform(fmap_flat)
        except Exception as e:
            print(f"PCA transform failed: {e}")
            return None

        fmap_pca = fmap_pca.T
        hogs = []
        
        winSize = (self.patch_size, self.patch_size)
        blockSize = (16, 16)
        blockStride = (8, 8)
        cellSize = (8, 8)
        nbins = 9
        hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

        for ch in fmap_pca:
            img = (ch - np.min(ch)) / (np.ptp(ch) + 1e-8)
            img = (img * 255).astype(np.uint8)
            img_resized_for_hog = cv2.resize(img, (self.patch_size, self.patch_size))
            
            hvec = hog.compute(img_resized_for_hog)
            if hvec is not None:
                hogs.append(hvec.flatten())
        
        if len(hogs) == 0: return None
        desc = np.concatenate(hogs)
        desc /= (np.linalg.norm(desc) + 1e-8)
        return desc

# ============ End Descriptor Class ============


# --------- RPE Helper (Bench Press Specific) ---------
def bench_velocity_to_rpe(velocity_mps: float) -> float:
    """
    Estimate RPE for bench press based on last-rep bar velocity (m/s).
    Source: VBT research (Sánchez-Medina & González-Badillo, 2011, etc.)
    """
    if velocity_mps >= 0.45:
        return 6.0    # ~RPE 6–7
    elif velocity_mps >= 0.35:
        return 7.0    # ~RPE 7–8
    elif velocity_mps >= 0.25:
        return 8.0    # ~RPE 8.5–9
    elif velocity_mps >= 0.15:
        return 9.0    # ~RPE 9–9.5
    else:
        return 10.0   # failure / grind


# --------- Calibration Helper ---------
plate_diameter_real = 0.45  # meters
points = []
scale = None
px_to_m = None
frame_resized = None # Declare globally for the callback

def click_event(event, x, y, flags, param):
    global points, frame_resized, scale, px_to_m
    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 2:
        points.append((x, y))
        cv2.circle(frame_resized, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Click Plate Diameter", frame_resized)

        if len(points) == 2:
            (x1, y1), (x2, y2) = points
            dist_px_resized = np.linalg.norm(np.array([x2-x1, y2-y1]))
            dist_px_original = dist_px_resized / scale
            px_to_m = plate_diameter_real / dist_px_original

            print(f"👉 Plate diameter in resized frame: {dist_px_resized:.1f} px")
            print(f"👉 Plate diameter in original frame: {dist_px_original:.1f} px")
            print(f"👉 Scale: {px_to_m:.6f} m/px")

            cv2.line(frame_resized, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.imshow("Click Plate Diameter", frame_resized)


# --------- Setup ---------
video_path = "bench2.mp4" # <-- Corrected video path
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video file: {video_path}")
    
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30
delay = int(1000 / fps)

# --- Calibration (Now 2 steps) ---
ret, frame = cap.read()
if not ret:
    raise RuntimeError("Could not read video for calibration")

# Resize for screen
max_w, max_h = 1280, 720
h, w = frame.shape[:2]
scale = min(max_w / w, max_h / h)
new_w, new_h = int(w * scale), int(h * scale)
frame_resized = cv2.resize(frame, (new_w, new_h))

# --- CALIBRATION STEP 1: COLOR (NEW) ---
print("--- CALIBRATION STEP 1: COLOR ---")
print("Are the plates COLORED (e.g., blue, red, yellow)? Press 'y' or 'n'.")
# Create a window to capture key press
cv2.imshow("Color Check", frame_resized)
key = cv2.waitKey(0) & 0xFF
is_colored = True if key == ord('y') else False
cv2.destroyWindow("Color Check")

print("Now, draw a box around the plate, then press ENTER.")
roi = cv2.selectROI("Select Plate (Draw Box)", frame_resized, fromCenter=False, showCrosshair=True)
cv2.destroyWindow("Select Plate (Draw Box)")

if roi[2] == 0 or roi[3] == 0:
    raise RuntimeError("Calibration failed. You must draw a box.")

roi_patch = frame_resized[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
hsv_roi = cv2.cvtColor(roi_patch, cv2.COLOR_BGR2HSV)
avg, std = cv2.meanStdDev(hsv_roi)

if is_colored:
    h_avg = avg[0][0]
    h_range = 15 # Hue range
    lower_hsv = np.array([max(0, h_avg - h_range), 40, 40]) # Min Sat and Val
    upper_hsv = np.array([min(179, h_avg + h_range), 255, 255])
    print(f"✅ COLOR mode. Hue center: {h_avg:.0f}, Range: [{lower_hsv[0]}, {upper_hsv[0]}]")
else:
    # Grayscale/Metallic mode. Filter by low Saturation and Value range.
    v_avg = avg[2][0]
    v_std = std[2][0]
    # Target low saturation (0-50) and a dynamic value range
    lower_hsv = np.array([0, 0, max(0, v_avg - 2*v_std)])
    upper_hsv = np.array([179, 50, min(255, v_avg + 2*v_std)])
    print(f"✅ GRAYSCALE mode. Value center: {v_avg:.0f}, Saturation: < 50")

print(f"Using HSV range: {lower_hsv} to {upper_hsv}")
# --- END COLOR CALIBRATION ---


# --- CALIBRATION STEP 2: SCALE (Original) ---
cv2.imshow("Click Plate Diameter", frame_resized)
cv2.setMouseCallback("Click Plate Diameter", click_event)
print("\n--- CALIBRATION STEP 2: SCALE ---")
print("Click on the top and bottom of a 45cm plate, then press any key.")
cv2.waitKey(0)
cv2.destroyWindow("Click Plate Diameter")

if px_to_m is None:
    raise RuntimeError("Calibration not done. Please click on the plate diameter.")
# --- END SCALE CALIBRATION ---


# --- Descriptor Setup ---
USE_DEEP_DESCRIPTOR = True  # This must be True to run

if USE_DEEP_DESCRIPTOR:
    print("Initializing DeepHOGDescriptor...")
    deep_descriptor = DeepHOGDescriptor(n_components=3, patch_size=64)
    
    # Use the same video, reset cap
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frames_for_pca = []
    for _ in range(10):
        ret, f = cap.read() # <-- FIXED: Use cap, not cap_tmp
        if not ret: break
        frames_for_pca.append(cv2.cvtColor(f, cv2.COLOR_BGR2GRAY))
    # <-- FIXED: Removed cap_tmp.release()
    
    if frames_for_pca:
        deep_descriptor.fit_pca(frames_for_pca)
    else:
        print("Error: Could not read frames to fit PCA.")
        USE_DEEP_DESCRIPTOR = False
        
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
else:
    raise RuntimeError("Deep descriptor is disabled — enable USE_DEEP_DESCRIPTOR = True to run.")

last_good_desc = None # Used for Pass 1 re-detection only

# --- HSV + Tracker Setup ---
# 'lower_hsv' and 'upper_hsv' are now set dynamically during calibration
tracker = None
tracking = False
bar_path = []
max_path_length = 40
last_center = None
frames_since_lost = 0
max_lost_frames = 5

# --- Kalman Filter Setup ---
kf = cv2.KalmanFilter(4, 2)
kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kf.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

# --- Analysis Variables ---
fps_list_pass1 = []
redetection_count_pass1 = 0
fps_list_pass2 = []
redetection_count_pass2 = 0
avg_fps = 0 # For on-screen display
MASTER_ANCHOR_DESC = None 

# --------- PASS 1: Backend Scan (lock thresholds) ---------
print("🔍 Backend scan running to prime state...")
# Reset video to start
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
frame_count = 0
bar_min_y, bar_max_y = None, None
in_bottom_position = False
rep_start_frame, rep_start_y = None, None
locked_min_y, locked_max_y = None, None
tracker, tracking, last_center, frames_since_lost, last_good_desc = None, False, None, 0, None

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret: break
    frame_count += 1
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bar_y = None

    if tracking:
        success, box = tracker.update(frame)
        if success:
            x, y, w_box, h_box = [int(v) for v in box]
            cx, cy = x + w_box // 2, y + h_box // 2
            roi = hsv[max(0, y):y + h_box, max(0, x):x + w_box]
            if roi.size == 0:
                frames_since_lost += 1
            else:
                mask_roi = cv2.inRange(roi, lower_hsv, upper_hsv)
                visible_ratio = cv2.countNonZero(mask_roi) / (w_box * h_box + 1e-6)
                if visible_ratio < 0.15: # ⚠️ You may also need to tune this visibility ratio
                    frames_since_lost += 1
                else:
                    frames_since_lost = 0
                    if w_box > 0 and h_box > 0:
                        roi_gray = gray[y:y + h_box, x:x + w_box]
                        if roi_gray.size > 0:
                            if USE_DEEP_DESCRIPTOR:
                                patch_resized = cv2.resize(roi_gray, (deep_descriptor.patch_size, deep_descriptor.patch_size))
                                desc = deep_descriptor.extract(patch_resized)
                                if desc is not None:
                                    last_good_desc = np.array([desc], dtype=np.float32)
                            
                    meas = np.array([[np.float32(cx)], [np.float32(cy)]])
                    kf.predict()
                    est = kf.correct(meas)
                    scx, scy = int(est[0]), int(est[1])
                    last_center = (scx, scy)
                    bar_y = scy

        if frames_since_lost > max_lost_frames:
            redetection_count_pass1 += 1
            tracking = False
            tracker = None
            frames_since_lost = 0

    if not tracking:
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        candidate_contours = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            # ⚠️ Tune these contour filters for your bench press plates
            if 300 < area < 8000: 
                peri = cv2.arcLength(cnt, True)
                if peri == 0: continue
                circ = 4 * np.pi * area / (peri ** 2)
                if 0.4 < circ < 1.3: # Check for circularity
                    candidate_contours.append((circ, cnt))
        best_contour = None
        if len(candidate_contours) == 1:
            best_contour = candidate_contours[0][1]
        elif len(candidate_contours) > 1 and last_good_desc is not None:
            max_matches = 0
            best_candidate = None
            for circ, cnt in candidate_contours:
                x_c, y_c, w_c, h_c = cv2.boundingRect(cnt)
                roi_c = gray[y_c:y_c + h_c, x_c:x_c + w_c]
                if roi_c.size == 0: continue
                
                desc_c = None 
                if USE_DEEP_DESCRIPTOR:
                    patch_resized_c = cv2.resize(roi_c, (deep_descriptor.patch_size, deep_descriptor.patch_size))
                    desc_c_vec = deep_descriptor.extract(patch_resized_c)
                    if desc_c_vec is not None:
                        desc_c = np.array([desc_c_vec], dtype=np.float32)

                if desc_c is not None and len(desc_c) > 0:
                    try:
                        matches = bf.match(last_good_desc, desc_c)
                        if len(matches) > max_matches:
                            max_matches = len(matches)
                            best_candidate = cnt
                    except cv2.error:
                        continue
            best_contour = best_candidate
        elif len(candidate_contours) > 1:
            candidate_contours.sort(key=lambda x: x[0], reverse=True)
            best_contour = candidate_contours[0][1]

        if best_contour is not None:
            x, y, w_box, h_box = cv2.boundingRect(best_contour)
            tracker = cv2.legacy.TrackerCSRT_create()
            tracker.init(frame, (x, y, w_box, h_box))
            tracking = True
            last_center = (x + w_box // 2, y + h_box // 2)
            bar_y = last_center[1]
            kf.statePost = np.array([[last_center[0]], [last_center[1]], [0], [0]], dtype=np.float32)
            roi_gray_init = gray[y:y + h_box, x:x + w_box]
            if roi_gray_init.size > 0:
                
                if USE_DEEP_DESCRIPTOR:
                    patch_resized_init = cv2.resize(roi_gray_init, (deep_descriptor.patch_size, deep_descriptor.patch_size))
                    desc_init = deep_descriptor.extract(patch_resized_init)
                    if desc_init is not None:
                        desc_init_arr = np.array([desc_init], dtype=np.float32)
                        last_good_desc = desc_init_arr # For Pass 1 re-detection
                        
                        # --- MODIFICATION: SAVE ANCHOR IMAGES ---
                        if MASTER_ANCHOR_DESC is None:
                            MASTER_ANCHOR_DESC = desc_init_arr
                            print("✅ Master Deep-HOG Anchor Descriptor has been captured.")
                            
                            try:
                                cv2.imwrite("anchor_patch_custom_64x64.jpg", patch_resized_init)
                                frame_with_box = frame.copy()
                                cv2.rectangle(frame_with_box, (x, y), (x + w_box, y + h_box), (0, 255, 0), 3)
                                cv2.putText(frame_with_box, "MASTER ANCHOR", (x, y - 10), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
                                cv2.imwrite("anchor_full_frame_custom.jpg", frame_with_box)
                                print("📸 Saved 'anchor_patch_custom_64x64.jpg' and 'anchor_full_frame_custom.jpg'")
                            except Exception as e:
                                print(f"Error saving anchor images: {e}")
                        # --- END MODIFICATION ---

    # --- Rep Logic (Pass 1) ---
    if bar_y is not None:
        if bar_min_y is None: bar_min_y, bar_max_y = bar_y, bar_y
        bar_min_y = min(bar_min_y, bar_y) # lockout
        bar_max_y = max(bar_max_y, bar_y) # chest
        range_y = bar_max_y - bar_min_y
        
        if range_y > 20: # ⚠️ Tune this pixel range if needed
            top_threshold = bar_min_y + 0.2 * range_y
            bottom_threshold = bar_max_y - 0.2 * range_y
            if bar_y > bottom_threshold:
                in_bottom_position = True
            elif in_bottom_position:
                rep_start_frame, rep_start_y = frame_count, bar_y
                in_bottom_position = False
            if rep_start_frame is not None and bar_y < top_threshold:
                rep_end_frame = frame_count
                rep_time = (rep_end_frame - rep_start_frame) / fps
                displacement_m = abs((rep_start_y - bar_y) * px_to_m)
                if rep_time > 0.1:
                    velocity = displacement_m / rep_time
                    rpe = bench_velocity_to_rpe(velocity) 
                    print(f"✅ Backend rep detected: Vel={velocity:.3f} m/s, RPE={rpe}")
                    locked_min_y, locked_max_y = bar_min_y, bar_max_y
                    break # Found anchor and thresholds, exit Pass 1
    
    # --- FPS Calc ---
    end_time = time.time()
    frame_time = end_time - start_time
    if frame_time > 0:
        fps_list_pass1.append(1 / frame_time)

# --- End of Pass 1 ---
cap.release() # Release cap after Pass 1 is done

if locked_min_y is None:
    print("⚠️ Warning: Could not detect a full rep. Rep counting may be inaccurate.")
    if bar_min_y is not None:
        locked_min_y, locked_max_y = bar_min_y, bar_max_y
    else:
        raise RuntimeError("No barbell detected in video at all.")

if MASTER_ANCHOR_DESC is None:
    print("⚠️ Warning: Could not capture a master anchor. Re-detection may be unstable.")

# --------- PASS 2: Main Playback ---------
print("🎥 Starting main playback...")
cap = cv2.VideoCapture(video_path) # Re-open cap for Pass 2
if not cap.isOpened():
    raise RuntimeError(f"Could not re-open video file for Pass 2: {video_path}")
    
rep_count, last_rep_velocity, est_rpe = 0, None, None
frame_count = 0
in_bottom_position = False
rep_start_frame, rep_start_y = None, None
tracker, tracking, last_center, frames_since_lost = None, False, None, 0
bar_path.clear()

while cap.isOpened():
    start_time = time.time()
    ret, frame = cap.read()
    if not ret: break
    frame_count += 1
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bar_y = None
    box_to_draw = None

    if tracking:
        success, box = tracker.update(frame)
        if success:
            x, y, w_box, h_box = [int(v) for v in box]
            box_to_draw = (x, y, w_box, h_box)
            cx, cy = x + w_box // 2, y + h_box // 2
            roi = hsv[max(0, y):y + h_box, max(0, x):x + w_box]
            if roi.size == 0:
                frames_since_lost += 1
            else:
                mask_roi = cv2.inRange(roi, lower_hsv, upper_hsv)
                visible_ratio = cv2.countNonZero(mask_roi) / (w_box * h_box + 1e-6)
                if visible_ratio < 0.15: # ⚠️ Tune this
                    frames_since_lost += 1
                else:
                    frames_since_lost = 0
                    meas = np.array([[np.float32(cx)], [np.float32(cy)]])
                    kf.predict()
                    est = kf.correct(meas)
                    scx, scy = int(est[0]), int(est[1])
                    last_center = (scx, scy)
                    bar_y = scy
                    bar_path.append(last_center)
                    if len(bar_path) > max_path_length: bar_path.pop(0)
                    cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)

        if frames_since_lost > max_lost_frames:
            redetection_count_pass2 += 1
            tracking = False
            tracker = None
            frames_since_lost = 0

    if not tracking:
        mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        candidate_contours = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if 300 < area < 8000: # ⚠️ Tune this
                peri = cv2.arcLength(cnt, True)
                if peri == 0: continue
                circ = 4 * np.pi * area / (peri ** 2)
                if 0.4 < circ < 1.3: # ⚠️ Tune this
                    candidate_contours.append((circ, cnt))
        best_contour = None
        if len(candidate_contours) == 1:
            best_contour = candidate_contours[0][1]
        
        elif len(candidate_contours) > 1 and MASTER_ANCHOR_DESC is not None:
            max_matches = 0
            best_candidate = None
            for circ, cnt in candidate_contours:
                x_c, y_c, w_c, h_c = cv2.boundingRect(cnt)
                roi_c = gray[y_c:y_c + h_c, x_c:x_c + w_c]
                if roi_c.size == 0: continue
                
                desc_c = None
                if USE_DEEP_DESCRIPTOR:
                    patch_resized_c = cv2.resize(roi_c, (deep_descriptor.patch_size, deep_descriptor.patch_size))
                    desc_c_vec = deep_descriptor.extract(patch_resized_c)
                    if desc_c_vec is not None:
                        desc_c = np.array([desc_c_vec], dtype=np.float32)

                if desc_c is not None and len(desc_c) > 0:
                    try:
                        matches = bf.match(MASTER_ANCHOR_DESC, desc_c)
                        if len(matches) > max_matches:
                            max_matches = len(matches)
                            best_candidate = cnt
                    except cv2.error:
                        continue
            best_contour = best_candidate
        elif len(candidate_contours) > 1:
            candidate_contours.sort(key=lambda x: x[0], reverse=True)
            best_contour = candidate_contours[0][1]
            
        if best_contour is not None:
            x, y, w_box, h_box = cv2.boundingRect(best_contour)
            box_to_draw = (x, y, w_box, h_box)
            tracker = cv2.legacy.TrackerCSRT_create()
            tracker.init(frame, (x, y, w_box, h_box))
            tracking = True
            last_center = (x + w_box // 2, y + h_box // 2)
            bar_y = last_center[1]
            kf.statePost = np.array([[last_center[0]], [last_center[1]], [0], [0]], dtype=np.float32)
            cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)

    # --- Draw Bar Path ---
    for i in range(1, len(bar_path)):
        if bar_path[i-1] is not None and bar_path[i] is not None:
            cv2.line(frame, bar_path[i - 1], bar_path[i], (0, 255, 255), 3)

    # --- Rep detection using locked thresholds ---
    if bar_y and locked_min_y is not None:
        top_threshold = locked_min_y + 0.2 * (locked_max_y - locked_min_y)
        bottom_threshold = locked_max_y - 0.2 * (locked_max_y - locked_min_y)

        if bar_y > bottom_threshold and not in_bottom_position:
            in_bottom_position = True
            rep_start_frame = None
        if in_bottom_position and bar_y < bottom_threshold:
            rep_start_frame, rep_start_y = frame_count, bar_y
            in_bottom_position = False
        if rep_start_frame is not None and bar_y < top_threshold:
            rep_end_frame = frame_count
            rep_time = (rep_end_frame - rep_start_frame) / fps
            if rep_time > 0.1:
                displacement_m = abs((rep_start_y - bar_y) * px_to_m)
                avg_velocity = displacement_m / rep_time
                # <-- FIXED: Call the correct RPE function
                last_rep_velocity, est_rpe = avg_velocity, bench_velocity_to_rpe(avg_velocity)
                rep_count += 1
                print(f"Rep {rep_count}: Vel={avg_velocity:.3f} m/s, RPE={est_rpe}")
            rep_start_frame = None

    # --- Display ---
    cv2.putText(frame, f"Bench Reps: {rep_count}", (50, 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

    if last_rep_velocity is not None:
        # est_rpe is already calculated during rep counting
        cv2.putText(frame, f"Last Rep Vel: {last_rep_velocity:.2f} m/s", (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.putText(frame, f"Est. RPE: {est_rpe}", (50, 200),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # --- FPS Calc & Display ---
    end_time = time.time()
    frame_time = end_time - start_time
    if frame_time > 0:
        current_fps = 1 / frame_time
        fps_list_pass2.append(current_fps)
        if len(fps_list_pass2) > 10:
            avg_fps = sum(fps_list_pass2[-10:]) / 10
            
    cv2.putText(frame, f"FPS: {avg_fps:.2f}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    
    # Resize for display
    h_frame, w_frame = frame.shape[:2]
    scale_disp = min(max_w / w_frame, max_h / h_frame)
    frame_display_resized = cv2.resize(frame, (int(w_frame * scale_disp), int(h_frame * scale_disp)))
    
    # <-- FIXED: Corrected window title
    cv2.imshow("Bench Press Tracking", frame_display_resized)
    if cv2.waitKey(delay) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

# 3. Print final report metrics
print("\n--- DEEP-HOG CV (ANCHOR): FINAL ANALYSIS ---")
if fps_list_pass1:
    print(f"Pass 1 (Scan) Avg FPS: {sum(fps_list_pass1) / len(fps_list_pass1):.2f}")
    print(f"Pass 1 (Scan) Re-Detections: {redetection_count_pass1}")
if fps_list_pass2:
    print(f"Pass 2 (Playback) Avg FPS: {sum(fps_list_pass2) / len(fps_list_pass2):.2f}")
    print(f"Pass 2 (Playback) Re-Detections: {redetection_count_pass2}")
print("------------------------------------\n")

--- CALIBRATION STEP 1: COLOR ---
Are the plates COLORED (e.g., blue, red, yellow)? Press 'y' or 'n'.
Now, draw a box around the plate, then press ENTER.
